# POLSCI 3 Fall 2021

## Week 8, Lecture Notebook 2: Generalizability

In [ ]:
library(estimatr)
library(ggplot2)

data <- read.csv("ps3_KB_campaign.csv")
head(data)

In this dataset, every row is an experiment. The authors (me and Josh Kalla) collected information from previous experiments and collected it in this dataset for meta-analysis.

Here is a quick rundown of what each column means:

- `Experiment`: Paper reference of the experiment
- `effect`: estimate of the effect size
- `SE`: standard error of the effect estimate
- `precision`: precision; this is how precise the experiment is. defined as $\frac{1}{\text{Standard Error}^2}$ (weights to be used)
- `General`: binary variable indicating whether the experiment was done in a general election (1 = yes, 0 = no)
- `Primary`: binary variable indicating whether the experiment was done in a primary election (1 = yes, 0 = no). these elections are typically quieter
- `Treatment.within.2months.of.election`: binary variable incating whether the experiment took place within 2 months of election day (1 = yes, was within 2 months; 0 = no, was more than 2 months before election day)

In [ ]:
# Looking at the first two entries (Arceneaux 2007), which of the two experiments are more precise? Why?
data[1:2,1:3]

In [ ]:
# How do we find the overall weighted effect?
weighted.effect.overall <- weighted.mean(data$effect, data$precision)
weighted.effect.overall

In [ ]:
# I'm going to create standard errors for you all.
simulations.overall <- c()
for (i in 1:1000) {
    temp.data <- data[sample(nrow(data), replace = TRUE),]
    temp.weighted.effect <- weighted.mean(temp.data$effect, temp.data$precision)
    simulations.overall <- append(simulations.overall, temp.weighted.effect)
}
standard.error.overall <- sd(simulations.overall)

In [ ]:
paste("The weighted average effect is", weighted.effect.overall)

paste("The standard deviation of the average effect is", standard.error.overall)

In [ ]:
# How do we find the weighted effect for experiments run within two months of election day?
two.months.within.election.day <- subset(data, data$Treatment.within.2months.of.election == 1)
head(two.months.within.election.day)

In [ ]:
weighted.effect.two.months.within.election.day <- ...
weighted.effect.two.months.within.election.day

In [ ]:
# How do we find the weighted effect for experiments run before two months from election day?
two.months.before.election.day <- subset(data, data$Treatment.within.2months.of.election == 0)
head(two.months.before.election.day)

In [ ]:
weighted.effect.two.months.before.election.day <- ...
weighted.effect.two.months.before.election.day

In [ ]:
#I'm going to simulate the two effects for you all
simulations.two.months.within <- c()
for (i in 1:1000) {
    temp.data <- two.months.within.election.day[sample(nrow(two.months.within.election.day), replace = TRUE),]
    temp.weighted.effect <- weighted.mean(temp.data$effect, temp.data$precision)
    simulations.two.months.within <- append(simulations.two.months.within, temp.weighted.effect)
}

simulations.two.months.before <- c()
for (i in 1:1000) {
    temp.data <- two.months.before.election.day[sample(nrow(two.months.before.election.day), replace = TRUE),]
    temp.weighted.effect <- weighted.mean(temp.data$effect, temp.data$precision)
    simulations.two.months.before <- append(simulations.two.months.before, temp.weighted.effect)
}


In [ ]:
#merge the simulations together and label them
plot.data <- data.frame(cbind(
    c(
        simulations.two.months.within,
        simulations.two.months.before
    ),
    c(
        rep("two months within", 1000),
        rep("two months before", 1000)
    )))
plot.data$X1 <- as.numeric(plot.data$X1)

In [ ]:
ggplot(plot.data, aes(x = X1, group = X2, fill = X2, alpha = 0.3)) +
    geom_density() +
    scale_x_continuous(limits = c(-1.5, 4))

In [ ]:
# to remind ourselves of the effect sizes:
paste("Treating people within two months of the election has an effect of", weighted.effect.two.months.within.election.day)

paste("Treating people two months before the election has an effect of", weighted.effect.two.months.before.election.day)